# Overview

This workbook explains the core priciples of the colourimetric based segmentation of microscope images, and ends with the classification of PD using segmentations of slide imagery.  It loads a number of basis functions for the Aperio L2 Microscope sensor, brain and staining functions, and processes them to produce likely RGB values found within the real imagery.  These RGB values in the real images are then estimated for their basis componenets by fitting them to the likely RGB values.  Likely candidate regions, those containing an amount of DAB stain, are segmented and saved as JPG files that can then be passed to a CNN developed previously (PDNet).

At the end of this workbook we will have a classifier that can efficiently and effectively detect PD pathology.

In [ ]:
# Change directory so 'Polygeist' is in the path.
import os

os.chdir("..")

import pathlib
import random
import shutil
import time

# OS & Utilities
from glob import glob

# Maths, Image, Science Libraries
import imageio.v3 as io
import numpy as np
from pqdm.processes import pqdm
from sklearn import metrics

# Polygeist libraries
import polygeist.colour as pc
from polygeist.slidecore.slide import (
    AperioSlide,
    SpectralSlideGenerator,
    SyntheticSlide,
)
from polygeist.training import train_model
from polygeist.utils import (
    calc_median_score_list,
    load_filenames_and_generate_conditions,
)

In [ ]:
# This loads matplotlib, a plotting library with the ability to display scalable plots in the browser.
%matplotlib widget
from matplotlib import pyplot as plt

In [ ]:
# We will display imagery relevant to the tutorial using this function, the files will be in a folder called 'assets'
def show_explainer_image(filename, title):
    im = io.imread(filename)
    plt.figure()
    plt.axis("off")
    plt.imshow(im)
    plt.title(title)
    plt.show()

# Photometric Calibration of the microscope

To effectively find the target protein (here a-syn) we need to understand how it appears to the microscope that captured the sample.  We can do this by determining the sensor response from our microscope, and then, given the integration of our stain transmission spectra, our lightsource, and our sensor sensitivity functions, we can determine the colour of the protein stained pixels.

Here we:
* Load the sensitivities of the sensor, the illumination power of the lightsource (by wl) and the transmission of the stains / matter under examination.
* Calculate the sensor response array, to be used to decompose a sensor response

In [ ]:
show_explainer_image(
    "notebook/assets/spectral_modelling.png",
    "Spectral Transmission Functions to Sensor RGB Response",
)

In [ ]:
from polygeist.microscope import Microscope

In [ ]:
ms = Microscope()

In [ ]:
# Dab Stain
DAB = pc.Illuminant.from_file_with_range("spectral/histology/DABL20Vector.csv", ms.wl)
# Hemotoxylin
H = pc.Illuminant.from_file_with_range("spectral/histology/HemotoxylinC19.csv", ms.wl)
# Eosin
E = pc.Illuminant.from_file_with_range("spectral/histology//Eosin6um.csv", ms.wl)
# Healthy Brain Absorption
_B = pc.Illuminant.from_file_with_range(
    "spectral/histology/HealthyBrainAbsorption9um.csv", ms.wl
)

In [ ]:
# Calculate the Transmission Function for the brain
_B[np.isnan(_B)] = 0
B = 1.0 - (_B / np.max(_B))

In [ ]:
# Produce the array of sensor RGB values for the given spectra
responses = np.vstack([ms.response(s) for s in [DAB, H, ms.ls, B, E]])
print(responses)

# These are sensor specific responses for the brain transmission * filter responses,
# for the AT2 sensor and lightsource.
# See reports for how to generate them for other sensing systems.

# 7.10357511 12.61506218 13.59695489  # DAB
# 8.81961536 10.18302502  5.08567669  # Hematoxylin
# 11.02074647 15.41804066 12.77042165 # Light Source
# 17.05035857 17.64819458  9.17788779 # Brain Transmission
# 0.45574971  4.32897163  1.68161384  # Eosin

## Sensor Response and Example Fit

The array above shows sensor BGR responses of the microscope sensor given the maximum brightness from the microscope lightsource and that surface in isolation.  Now we are going to use these values in an example spectral fit.  We will load an example slide file (or use simulated data), and find pixels that contain DAB stain.

In [ ]:
show_explainer_image(
    "notebook/assets/spectral_fitting.png",
    "Process of Decomposing a Pixel into Spectral Weights",
)

In [ ]:
# Load the data or use simulated data
simulated_data = True

In [ ]:
# Put your path here if you are not using simulated data!
example_slide_path = "localnas/A-syn cases/PD/PD788/PD788-17_A-syn.svs"

if simulated_data:
    image = SpectralSlideGenerator(width=100, height=100).image
else:
    image = AperioSlide(example_slide_path).get_slide_with_pixel_resolution_in_microns(
        2.0
    )

In [ ]:
plt.figure()
plt.imshow(image.astype(int))
plt.title("Slide Image Containing Pixel Data of Tissue and Stain")
plt.show()

In [ ]:
# define the normalisation function we will use to scale within our planes
def normalise(F):
    F += max(np.abs(F.min()), np.abs(F.max()))
    F /= F.max()
    return F

In [ ]:
# Convert our image to Nx3 Tristimulus values
T_I = image.reshape((image.shape[0] * image.shape[1], 3))

# Least squares fit to our response functions
T_x = np.linalg.lstsq(responses.T, T_I.T, rcond=None)

# Normalise our responses per map for DAB and Brain Transmission
DAB = normalise(T_x[0][0].copy())
BT = normalise(T_x[0][3].copy())

raw_DAB = DAB - BT
raw_DAB = raw_DAB.reshape(image.shape[0], image.shape[1])

_DAB = T_x[0][0].reshape(image.shape[0], image.shape[1])
_BT = T_x[0][3].reshape(image.shape[0], image.shape[1])

# Values below our raw threshold will be considered A-syn (typically BT has significantly more power)
raw_threshold = -0.3

In [ ]:
# Show our decomposition
fig, axs = plt.subplots(2, 2)

fig.suptitle("Comparisons between Original, Decomposed & Isolated Signals")
axs[0, 0].set_title("RGB Image")
axs[0, 0].imshow(image.astype(int))
axs[0, 1].set_title("DAB Estimate")
axs[0, 1].imshow(normalise(_DAB))
axs[1, 0].set_title("Brain Transmission Estimate")
axs[1, 0].imshow(normalise(_BT))
axs[1, 1].set_title("Asyn Isolation")
# Here we threshold the raw DAb signal to get A-syn
axs[1, 1].imshow(raw_DAB < raw_threshold)

# Example Segmentation of Images Using the Microscope Calibration

Above you can see the decomposed channels for each of the proteins, and the asyn isolation channel is a boolean mask, where we have marked each pixel that exceeds a threshold and highlighted it as a DAB pixel.

Now we can use this technology in a segmentation routine, here we will traverse our slides (or simulated data) and with a given window size, we will segment those windows into the same boolean channel as can be seen above in 'Asyn Isolation'.  We will then check that window to see what the mean activation (density of pixels are 'on' (yellow above)) and then write our regions to disk that exceed that threshold.  We will also save the density information into a 'density map' for later use.

We will do this for both the PD and Control groups, and then use those segmentation to train a classifier, as the control group should be exclusively false alarms.  

In [ ]:
show_explainer_image(
    "notebook/assets/density_map_generation.png",
    "Turning Pathology Slides into Density Maps and ROIs",
)

In [ ]:
config = {
    # The threshold level in RGB to use for our prexisting RGB thresholding routines from Phase 1
    "B_channel_threshold": 20,
    # This is where we will be storing our segmeneted data
    "dump_path": "/run/media/brad/ScratchM2/maps_dump_512_jpeg/",
    # This is our window size, over which we will iterate.
    "map_stride": 512,
    # Our negative cases, our controls, are stored in the 'Controls' folder
    "negative_case_folder": "Controls",
    # Our positive, pd cases, are stored in the 'PD' folder
    "positive_case_folder": "PD",
    # Other search paths to look for cases
    "search_directories": [
        "/run/media/brad/TBSSD1/A-syn cases/",
        "/run/media/brad/TBSSD2/A-syn cases/",
    ],
    # These are our case filenames
    "case_files": "Data/filenames/asyn_files.txt",
    # For use with real data, this filters slides with the wrong index
    "slide_index_filter": "17_A",
    # Toggle this is you are using simulated data
    "simulated": simulated_data,
    # Number to simulate
    "simulation_n": 4,
}

In [ ]:
def spectral_decompose_and_dump_jpeg(
    slice,
    stride=1000,
    process_raw=True,
    threshold=0.175,
    pc=0.0375,
    raw_threshold=-0.3,
    raw_pc=0.00125,
    jpeg_dump_path_and_name="start_name",
):

    # These are sensor specific responses for the brain transmission * filter responses, for the AT2 sensor
    # and lightsource.  See reports for how to generate them for other sensing systems
    responses = np.array(
        np.matrix(
            "[ 7.10357511 12.61506218 13.59695489 ; "  # DAB
            "8.81961536 10.18302502  5.08567669; "  # Hematoxylin
            "11.02074647 15.41804066 12.77042165 ; "  # Light Source
            "17.05035857 17.64819458  9.17788779; "  # Brain Transmission
            "0.45574971  4.32897163  1.68161384]"
        )
    )  # Eosin

    # Get slide at native resolution
    image = slice.get_slide_with_pixel_resolution_in_microns(2.0)

    # Get the height and width of the slice
    yy, xx, _ = image.shape

    # Create a densities array to store the local densities
    x_pass = int(np.ceil(xx / stride))
    y_pass = int(np.ceil(yy / stride))
    densities = np.zeros((y_pass, x_pass))

    # Convert our image to Nx3 Tristimulus values
    T_I = image.reshape((image.shape[0] * image.shape[1], 3))

    # Least squares fit to our response functions
    T_x = np.linalg.lstsq(responses.T, T_I.T, rcond=None)

    # Normalise our reponses per map for DAB and Brain Transmission
    DAB = normalise(T_x[0][0].copy())
    BT = normalise(T_x[0][3].copy())

    # Remove the background (Brain) fom DAB response, leaving just the dab mask
    # Remove the background (Brain) fom DAB response, leaving just the dab mask
    raw_DAB = DAB - BT
    raw_DAB = raw_DAB.reshape(image.shape[0], image.shape[1])

    # ticker for JPEG index
    dump_number = 0
    if not process_raw:
        # Tumble over the slice using a fixed window size
        for xi, x in enumerate(np.arange(0, xx, stride)):
            for yi, y in enumerate(np.arange(0, yy, stride)):
                # Grab this section x -> x + stride, y -> y + stride
                section = raw_DAB[y : y + stride, x : x + stride]

                diff = np.abs(
                    section[:, :, 0].astype(float) - section[:, :, 2].astype(float)
                )
                if np.sum(diff > threshold) / (stride**2) > pc:
                    io.imwrite(
                        f"{jpeg_dump_path_and_name}{dump_number}.jpg",
                        image[y : y + stride, x : x + stride, :],
                    )
                    dump_number += 1
    else:
        # Tumble over the slice using a fixed window size
        for xi, x in enumerate(np.arange(0, xx, stride)):
            for yi, y in enumerate(np.arange(0, yy, stride)):
                # Grab this section x -> x + stride, y -> y + stride
                section = raw_DAB[y : y + stride, x : x + stride] < raw_threshold

                if np.mean(section) > raw_pc:
                    io.imwrite(
                        f"{jpeg_dump_path_and_name}{dump_number}.jpg",
                        image[y : y + stride, x : x + stride, :],
                    )
                    dump_number += 1
    return densities

In [ ]:
# This process function will be called in parallel on each slide.
def process(argv):
    file = argv["file"]
    cnd = argv["condition"]
    simulated = argv["sim"]

    try:
        file_name = os.path.basename(os.path.normpath(file))
        slide = AperioSlide(file) if not simulated else SyntheticSlide(file)
    except:
        return file
    d = spectral_decompose_and_dump_jpeg(
        slide,
        stride=config["map_stride"],
        jpeg_dump_path_and_name=f'{config["dump_path"]}/{cnd}/{file_name}',
    )
    with open(f'{config["dump_path"]}/{cnd}/{file_name}.npy', "wb") as f:
        np.save(f, d)

    return file

In [ ]:
# Make the directory structure to save our files
for pth in [config["positive_case_folder"], config["negative_case_folder"]]:
    pathlib.Path(config["dump_path"] + pth).mkdir(parents=True, exist_ok=True)

files_list_and_configuration = []
# For each case in positive/negative case directories, or for simulated data
if not config["simulated"]:
    for cases in [config["positive_case_folder"], config["negative_case_folder"]]:
        for searched_path in config["search_directories"]:
            for p in glob(searched_path + cases + "/*/", recursive=True):
                for f in glob(p + "/*.svs", recursive=True):
                    if config["slide_index_filter"] in f:
                        files_list_and_configuration.append(
                            {
                                "file": f,
                                "condition": cases,
                                "sim": False,
                            }
                        )
else:
    for i in np.arange(0, config["simulation_n"]):
        # Which condition to generate, PD or Control
        condition = (
            config["positive_case_folder"]
            if np.random.random() > 0.5
            else config["negative_case_folder"]
        )
        # We will use the dataset name conv for our slides.
        name_for_slide = "PD" if "PD" in condition else "PDC"
        # Generate a new slide and save it to disk
        SpectralSlideGenerator(
            512 * 4,
            512 * 4,
            filename=f"{config['dump_path']}/{name_for_slide}{i}-17_A.png",
            control="C" in name_for_slide,
        )
        # Append and continue
        files_list_and_configuration.append(
            {
                "file": f"{config['dump_path']}/{name_for_slide}{i}-17_A.png",
                "condition": condition,
                "sim": True,
            }
        )

result = pqdm(
    files_list_and_configuration, process, n_jobs=1
)  # 1 job for AJAX optimistion

In [ ]:
case_conditions = load_filenames_and_generate_conditions(config["case_files"])

positive_cases = []
negative_cases = []
for case, condition in case_conditions.items():
    positive_cases.append(case) if "C" not in condition else negative_cases.append(case)

# Raw Analysis of Density Maps

Let's first just take a look at the density maps that have been dumped.  We are going to use the basic thresholding routines to determine if we have successfully segmented asyn, before we look into more advanced classifications methods in the other WPs.

Process:

* Create histograms of the density for each case, and band pass those densities to highlight the maximal differences between the groups.
* Calculate the mean bandpass density for each 'case', and then produce an ROC on the basis of those densities.

In [ ]:
# Here we use a simple median classifier, with a low RGB threshold as a simple mask.
# Note this is for illustrative purposes,
# we are going to explore a better classification method in the following workbooks.

# Count results of simple median classifier
positive_score_list = calc_median_score_list(
    positive_cases,
    f'{config["dump_path"]}/{config["positive_case_folder"]}/*17_*.jpg',
    rgb_threshold=config["B_channel_threshold"],
)

negative_score_list = calc_median_score_list(
    negative_cases,
    f'{config["dump_path"]}/{config["negative_case_folder"]}/*17_*.jpg',
    rgb_threshold=config["B_channel_threshold"],
)

In [ ]:
# Plot the median Counts
fig, ax = plt.subplots()
ax.boxplot([negative_score_list, positive_score_list], showfliers=False)
ax.set_xticks([1, 2], ["Control", "PD"])
plt.xlabel("Condition")
plt.ylabel("Quartiles of ROIs Identified")
plt.title("Interquartile Ranges for Each Case's Median Density")

In [ ]:
labels = np.hstack(
    [np.ones(len(positive_score_list)), np.zeros(len(negative_score_list))]
)
outputs = np.hstack([positive_score_list, negative_score_list])

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(labels, outputs)

In [ ]:
plt.figure()
plt.plot(fpr, tpr, label="PD vs Control")
plt.legend()
plt.xlabel("False Alarm Rate", fontsize=18)
plt.ylabel("Hit Rate", fontsize=18)
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)
# plt.title("512um Patch Level Discrmination between Taupathology and Control Tau Segmentation")
plt.show()

# Machine Learning with PDNet

Above we can see fair classifcation based on a single statistics, the median density from the ROI maps.  Now we are going to take those segmented ROIs, and use them to train PDNet.  We will:
   - Split the ROIs into training and test sets
   - Use the training set to train PDNet
   - Evaluate the per ROI classification accuracy of PDNet using the confidence scores from the validation set

In [ ]:
show_explainer_image(
    "notebook/assets/pdnet_training.png", "Schematic of Training PDNet on ROIs"
)

## Dataprep
Here we ready the folders for our model output, and our training and test data.  For real segmentations, we will balance our sets using randomly sampled real slide windows (non-ROIs).  This will be skipped for simulated data.

In [ ]:
# Our dump path for our model training run, model checkpoints will be saved here
model_dump_dir = f"{config['dump_path']}/model_dump"
batch_size = 32  # Adjust for memory constraints (may effect results)
num_epochs = 500  # Adjust for time available for training (may effect results)

# Note, if training fails even with small memory requirements, you may need to restart the kernel
# (The staining model may not have released its memory.)

In [ ]:
# Now we have our folders, we need to create a training and validation set.
# We will use a clean copy of the data for performance, repeatability and safety.
training_dump_path = f"{config['dump_path']}/training_dump"

In [ ]:
# Make the directory structure to save our files
for pth in [training_dump_path, model_dump_dir]:
    pathlib.Path(pth).mkdir(parents=True, exist_ok=True)

In [ ]:
# Make the directory structure to save our files
for pth in [training_dump_path + "/train/", training_dump_path + "/val/"]:
    for s in ["Controls", "PD"]:
        pathlib.Path(pth + s).mkdir(parents=True, exist_ok=True)

In [ ]:
# Should we copy files to create training dataset?
# Skip this cell if the data has already been prepared
# Do not run twice as it does not remove old datasets.
skip = len(glob(f"{training_dump_path}/train/Controls/*.jpg")) > 0

if not skip:
    # Splits
    prop_data_train = 0.6

    # Copy and partition the files (train and val)
    for s in ["Controls", "PD"]:
        for file in glob(f"{config['dump_path']}/{s}/*.jpg"):
            # basename for dumping out
            base = os.path.basename(file)
            f = io.imread(file)
            if f.shape[0] == f.shape[1]:
                if random.random() > (1.0 - prop_data_train):
                    shutil.copyfile(file, f"{training_dump_path}/train/{s}/" + base)
                else:
                    shutil.copyfile(file, f"{training_dump_path}/val/{s}/" + base)

# Now we are going to balance the training and test sets, to prevent overfitting, we do this by randomly sampling new
# regions from the control set images until the number of control squares is the same of the test squares.
# !! NOTE !! This only works when set A > set B.  If set B was larger, we would have to get more examples from set A
# where a random sample would not work.
l_control = len(glob(f"{training_dump_path}/train/Controls/*.jpg"))
l_test = len(glob(f"{training_dump_path}/train/PD/*.jpg"))

In [ ]:
if not config["simulated"]:
    from polygeist.slidecore import AperioSlide as Slide

    # list of files in control list
    control_file_list = glob(f"{config['search_directories'][0]}/Controls/*/*.svs")

    # Continue while set is unbalanced ~100
    control_injection_index = 0
    while l_test > l_control:
        # Randomise the list
        random.shuffle(control_file_list)

        # Sample the top of the list
        slide = Slide(control_file_list[0]).get_slide_with_pixel_resolution_in_microns(
            2.0
        )
        filename = os.path.basename(control_file_list[0])

        yy, xx, _ = slide.shape

        # Create a densities array to store the local densities
        x_pass = int(np.ceil(xx / 512))
        y_pass = int(np.ceil(yy / 512))

        for x, y in zip(
            np.random.randint(0, x_pass - 1, 25), np.random.randint(0, y_pass - 1, 25)
        ):
            im = slide[
                (y * 512) : (y * 512) + 512, (x * 512) : (x * 512) + 512, :
            ].copy()
            io.imwrite(
                f"{training_dump_path}/train/Controls/CI_{filename}_{control_injection_index}.jpg",
                im,
            )
            control_injection_index += 1

        # Recount
        l_control = len(glob(f"{training_dump_path}/train/Controls/*.jpg"))
        l_test = len(glob(f"{training_dump_path}/train/PD/*.jpg"))

## Training
Now that we have our training set in place, we can call our `train_model` function which will handle the training.

In [ ]:
# We don't inject into the validation set, that is kept clean for validation of the colourimetric segmentation.

# Start a timer
start_time = time.time()

latest_model_name = train_model(
    training_dump_path, model_dump_dir, batch_size, num_epochs, strict=False
)

time_elapsed = time.time() - start_time
print(f"Training complete in {time_elapsed // 60}m {time_elapsed % 60}s")

## Validation
Here, we use the output of the model, which is stored in `model_dump_dir`.  A hard coded the model used during development can be found below.  We will load this model, and then pass our validation set to it, and generate ROC curves. 

In [ ]:
### NOTE: You will need to put your latest model file here
model_path = "//path/to/model"

In [ ]:
import torch
from PIL import Image
from torchvision import transforms

from polygeist.CNN import PDNet

In [ ]:
# Create the network and ship it to the graphics card, use the output model file
model = PDNet()
model.apply_state(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model = model.eval()

In [ ]:
# We will implement transforms to get the images into the network like in the training
transform_for_input = transforms.Compose(
    [
        transforms.Resize(299),
        transforms.ToTensor(),
    ]
)

In [ ]:
# Go through and classify all the images
results = {}
for s in ["Controls", "PD"]:
    for file in glob(f"{training_dump_path}/val/{s}/*.jpg"):
        with Image.open(file) as im:
            f = transform_for_input(im).to(device)
            with torch.set_grad_enabled(False):
                results[file] = model(f.unsqueeze(0))

In [ ]:
# First - Patch level discrimination
outputs = []
labels = []
for key, value in results.items():
    outputs.append(float(value))
    labels.append(1.0 if "C" in key else 0.0)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(labels, 1.0 - np.array(outputs))

## Per ROI Classification Accuracy

Here we evaluate the per patch accuracy for classifying PD and Control.

In [ ]:
plt.figure()
plt.plot(fpr, tpr, label="PD vs Control")
plt.legend()
plt.xlabel("False Alarm Rate", fontsize=18)
plt.ylabel("Hit Rate", fontsize=18)
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)
plt.show()

# Conclusions

In this workbook we have started with slides, digitised samples of brain tissue from either control brains or those with PD pathology, and we have:
   - Identified likely regions of interest using a spectral estimation technique to find DAB stain.
   - Segmented those ROIs into a size that can be processed by ML algorithms.
   - Partitioned those data into training and test sets.
   - Trained a CNN (PDNet) to discriminate those ROIs
   - Produced a validation ROC curve demonstrating performance on unseen ROIs